In [1]:
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import concurrent.futures
import pandas as pd
import re
import requests


def get_soup(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    return soup

In [2]:
def get_seasons(url):
    soup = get_soup(url)
    
    seasons = soup.find('select', {'name': 'saison_id'}).find_all('option')
    
    ssn_list = []
    for season in seasons:
        season_id = season['value'].strip()
        season_name = season.text.strip()
        ssn_list.append({
            'season': season_name,
            'season_id': season_id
        })
    return ssn_list

club_url = 'https://www.transfermarkt.us/tranmere-rovers/kader/verein/1074/saison_id/2024/'

seasons = get_seasons(club_url)

seasons[:3]

[{'season': '24/25', 'season_id': '2024'},
 {'season': '23/24', 'season_id': '2023'},
 {'season': '22/23', 'season_id': '2022'}]

In [3]:
def get_ssn_url(season_id):
    return f'https://www.transfermarkt.us/tranmere-rovers/kader/verein/1074/saison_id/{season_id}'

In [4]:
def get_players(soup):
    return soup.find_all(class_='posrela')

def get_player_link(player):
    p = player.find(class_='hauptlink').find('a')
    player_name = p.text.strip()
    player_url = p['href']
    return {
        'player_name': player_name,
        'player_url': player_url
    }

# Get player names and URLs for every season

In [5]:
def process_season(season):
    season_id = season['season_id']
    season_url = get_ssn_url(season_id)
    soup = get_soup(season_url)
    players = get_players(soup)
    
    player_links = []
    for player in players:
        player_link = get_player_link(player)
        player_links.append(player_link)
    return player_links

plr_list = []

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_season, s) for s in seasons]
    
    for future in tqdm(concurrent.futures.as_completed(futures)):
        plr_list.extend(future.result())

0it [00:00, ?it/s]

In [6]:
plr_list = pd.DataFrame(plr_list).drop_duplicates().to_dict(orient='records')

plr_list[:3]

[{'player_name': 'Iain Turner',
  'player_url': '/iain-turner/profil/spieler/3582'},
 {'player_name': 'Scott Davies',
  'player_url': '/scott-davies/profil/spieler/67273'},
 {'player_name': 'Paddy Wharton',
  'player_url': '/paddy-wharton/profil/spieler/504952'}]

# Get HTML content of player profile

In [7]:
def process_player(plr):
    plr_url = f"https://www.transfermarkt.us{plr['player_url']}"
    plr_name = plr['player_name']
    plr_soup = get_soup(plr_url)
    return {
        'player_name': plr_name,
        'player_url': plr_url,
        'player_html': plr_soup
    }

plrs_html = []

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_player, plr) for plr in plr_list]
    
    for future in tqdm(concurrent.futures.as_completed(futures)):
        plrs_html.append(future.result())

0it [00:00, ?it/s]

# Pull player positions from player html

In [8]:
df = pd.DataFrame(plrs_html)

def get_position(soup):
    positions = soup.find_all('dd', class_='detail-position__position')
    return [pos.text.strip() for pos in positions]

df['positions'] = df['player_html'].apply(get_position)

df['positions'][:3]

0    [Centre-Back, Right-Back, Defensive Midfield]
1                        [Centre-Back, Right-Back]
2                                     [Goalkeeper]
Name: positions, dtype: object

# Fix names pulled from Transfermarkt to match names in main dataset

In [9]:
name_fixes = {
    'Ken Beamish': 'Kenny Beamish',
    'Dave Burgess': 'David Burgess',
    'Jim Cassidy': 'James Cassidy',
    'James Cumbes': 'Jim Cumbes',
    'Ron Dellow': 'Ronnie Dellow',
    'Stanley Docking': 'Stan Docking',
    'Herbert Hamilton': 'Duke Hamilton',
    'Jim Harvey': 'Jimmy Harvey',
    'Steve Jennings': 'Steven Jennings',
    'Ousmane Kane': 'Ousmane Kané',
    'Matty Kennedy': 'Matthew Kennedy',
    'John King': 'Johnny King',
    'Nathaniel Knight-Percival': 'Nat Knight-Percival',
    'Shay Logan': 'Shaleum Logan',
    'Jonathon Margetts': 'Johnny Margetts',
    'Hugh McAuley': 'Hughie McAuley',
    'Jay McEveley': 'James McEveley',
    'Manny Monthe': 'Emmanuel Monthe',
    'John Morrissey': 'Johnny Morrissey',
    'Joe Starbuck': 'Joseph Starbuck',
    'James Steel': 'Jim Steel',
    "Steven O'Leary": "Stephen O'Leary",
    'Sam Taylor': 'Samuel Taylor',
    'Danny Woodards': 'Dan Woodards',
}

for name in name_fixes:
    df.loc[df['player_name'] == name, 'player_name'] = name_fixes[name]

# Add player positions to dataframe

In [10]:
# Find the maximum length of the lists in 'positions' column
max_len = df['positions'].apply(len).max()

# Create new columns 'position_1', 'position_2', etc.
for i in range(max_len):
    df[f'tm_pos_{i+1}'] = df['positions'].apply(lambda x: x[i] if i < len(x) else None)

# Drop the original 'positions' column
df = df.drop(columns=['positions'])

# Pull players' birth dates from Transfermarkt HTML

In [11]:
def get_dob(soup):
    soup = str(soup)
    pattern = r'\d{4}-\d{2}-\d{2}'
    match = re.search(pattern, soup)
    if match:
        return match.group()
    else:
        return None
    
df['player_dob'] = df['player_html'].apply(get_dob)

# Fix incorrect dates of birth found in Transfermarkt data

In [12]:
dob_fixes = {
    'Alex Woodyard': '1993-05-03',
    'Callum Lucy': '1998-11-09',
    'Christian Edwards': '1975-11-23',
    'Eddie Bishop': '1962-11-28',
    'Edgar Walkden': '1914-11-04', # Complete Record
    'Elliot Osborne': '1996-05-12', # Unclear, but most common
    'Ernie Davies': '1916-01-31',
    'Gilbert Wassell': '1910-04-09',
    'Gordon West': '1943-04-24',
    'James Norwood': '1990-10-24',
    'John Aldridge': '1958-09-18',
    'John Griffiths': '1916-06-30', # Complete Record
    'John Williams': '1960-10-03',
    'Kane Hemmings': '1991-04-08',
    'Neil Gibson': '1979-10-10', # Complete Record
    'Oliver James': '1987-01-13', # Official site Wayback Machine
    'Rob Apter': '2003-04-23',
    'Robbie Burns': '1990-11-15', # LCFC Wayback Machine
    'Ronnie Moore': '1953-01-29',
    'Ryan Edwards': '1993-10-07',
    'Ryan Shotton': '1988-10-30',
    'Samuel Taylor': '2003-12-23',
    'Scott Davies': '1987-02-23',
    "Seyni N'Diaye": '1973-06-01',
    "Stephen O'Leary": '1987-02-02',
    'Ted Buckley': '1912-09-13',
    'Will Aimson': '1994-06-03',
}

for name in dob_fixes:
    df.loc[df.player_name==name, 'player_dob'] = dob_fixes[name]

In [13]:
pos_df = df.drop(columns=['player_url', 'player_html']).copy()

In [14]:
pos_df

,player_name,tm_pos_1,tm_pos_2,tm_pos_3,player_dob
0,Mitch Duggan,Centre-Back,Right-Back,Defensive Midfield,1997-03-20
1,Michael Ihiekwe,Centre-Back,Right-Back,None,1992-11-20
2,Paddy Wharton,Goalkeeper,None,None,2000-05-27
3,Evan Gumbs,Centre-Back,None,None,1997-07-21
4,Ritchie Sutton,Centre-Back,Right-Back,None,1986-04-29
...,...,...,...,...,...
582,Eddie Loyden,Centre-Forward,None,None,1945-12-22
583,Tommy Young,Centre-Forward,Attacking Midfield,None,1947-12-24
584,Willie Stevenson,Central Midfield,Centre-Back,Left Midfield,1939-10-26
585,Syd Farrimond,Left-Back,None,None,1940-07-17


# Split names to create forename and surname columns

In [15]:
def split_name(name):
    if name == 'Pedro Miguel Matias':
        forename = 'Pedro Miguel'
        surname = 'Matias'
    else:
        parts = name.split()
        forename = parts[0]
        surname = ' '.join(parts[1:])
    return pd.Series([forename, surname])
    
pos_df[['forename', 'surname']] = pos_df['player_name'].apply(split_name)

pos_df = pos_df.sort_values(['surname', 'forename', 'player_dob'])

# Fix errors in Transfermarkt player positions

In [16]:
pos_fixes = {
    'Lucas Akins': ['Right Winger', 'Centre-Forward'],
    'Graham Allen': ['Centre-Back', 'Right-Back'],
    'Doug Anderson': ['Winger'],
    'John Aspinall': ['Winger'],
    'Michael Black': ['Left Winger', 'Right Winger'],
    'Paul Black': ['Left-Back', None],
    'Ivano Bonetti': ['Right Winger', 'Left Winger'],
    'Ged Brannan': ['Central Midfield', 'Left-Back'],
    'Paul Brown': ['Left Midfield'],
    'Shane Cansdell-Sherriff': ['Left-Back', 'Centre-Back'],
    'Logan Chalmers': ['Left Winger', 'Right Winger', None],
    'Paul Cook': ['Central Midfield', 'Left Midfield'],
    'Tom Curtis': ['Central Midfield'],
    'Janoi Donacien': ['Centre-Back', 'Right-Back', None],
    'Sean Flynn': ['Central Midfield'],
    'Stephen Frail': ['Right-Back', 'Defensive Midfield'],
    'Wayne Gill': ['Attacking Midfielder'],
    'Jermaine Grandison': ['Centre-Back'],
    'Jimmy Harvey': ['Central Midfield'],
    'Alex Hay': ['Centre-Forward', 'Right Winger'],
    'Dave Higgins': ['Centre-Back'],
    'Matt Hill': ['Left-Back', None],
    'Mark Hughes': ['Centre-Back', 'Sweeper'],
    'Gary Jones': ['Central Midfield', 'Striker', 'Centre-Back'],
    'Dave Martindale': ['Central Midfield', 'Defensive Midfield'],
    'Pedro Miguel Matias': ['Left Winger'],
    'Micky Mellon': ['Central Midfield'],
    "Seyni N'Diaye": ['Center-Forward'],
    'James Olsen': ['Left-Back', 'Left Midfield'],
    'Elliot Osborne': ['Central Midfield'],
    'Andy Thompson': ['Left-Back'],
    'Ryan Williams': ['Attacking Midfielder'],
    'Steve Yates': ['Centre-Back', 'Right-Back', 'Left-Back']
}

for player, positions in pos_fixes.items():
    for i, pos in enumerate(positions):
        pos_df.loc[pos_df.player_name==player, f'tm_pos_{i+1}'] = pos

pos_df = pos_df[['surname', 'forename',  'player_name', 'player_dob', 'tm_pos_1', 'tm_pos_2', 'tm_pos_3']]

pos_df.head(5)

,surname,forename,player_name,player_dob,tm_pos_1,tm_pos_2,tm_pos_3
233,Achterberg,John,John Achterberg,1971-07-08,Goalkeeper,None,None
495,Adkins,Nigel,Nigel Adkins,1965-03-11,Goalkeeper,None,None
249,Ahmed,Adnan,Adnan Ahmed,1984-06-07,Central Midfield,None,None
92,Aimson,Will,Will Aimson,1994-06-03,Centre-Back,None,None
349,Aiston,Sam,Sam Aiston,1976-11-21,Left Midfield,None,None


In [17]:
import re

def view_special_chars(df, col):
    pattern = re.compile(r'[^\x00-\x7F]+')
    return df[df[col].str.contains(pattern)]

view_special_chars(pos_df, 'player_name')

,surname,forename,player_name,player_dob,tm_pos_1,tm_pos_2,tm_pos_3
361,Carole,Sébastien,Sébastien Carole,1982-09-08,Right Winger,Attacking Midfield,Right Midfield
322,Dadi,Eugène,Eugène Dadi,1973-08-20,Centre-Forward,None,None
120,Fôn Williams,Owain,Owain Fôn Williams,1987-03-17,Goalkeeper,None,None
301,Golobart,Román,Román Golobart,1992-03-21,Centre-Back,None,None
352,Gulácsi,Péter,Péter Gulácsi,1990-05-06,Goalkeeper,None,None
134,Kané,Ousmane,Ousmane Kané,2001-07-23,Defensive Midfield,Central Midfield,None
302,Sidibé,Mamady,Mamady Sidibé,1979-12-18,Centre-Forward,None,None
21,Sousa,Érico,Érico Sousa,1995-03-12,Left Winger,Right Winger,None
226,Traoré,Drissa,Drissa Traoré,1992-03-25,Central Midfield,Defensive Midfield,None


In [18]:
def replace_special_chars(name):
    name = name.replace('á', 'a')
    name = name.replace('é', 'e')
    name = name.replace('é', 'e')
    name = name.replace('è', 'e')
    name = name.replace('É', 'E')
    name = name.replace('ô', 'o')
    return name

# Create `player_id` column

In [19]:
def create_player_id(row):
    forename = row['forename']
    forename = replace_special_chars(forename)
    surname = row['surname']
    surname = replace_special_chars(surname)
    dob = row['player_dob']
    player_id = f'{surname}{forename}{dob}'.replace('.0', '').replace('-', '').replace("'", '').replace(' ', '').replace('None', '')
    return player_id

pos_df['player_id'] = pos_df.apply(create_player_id, axis=1)

pos_df

,surname,forename,player_name,player_dob,tm_pos_1,tm_pos_2,tm_pos_3,player_id
233,Achterberg,John,John Achterberg,1971-07-08,Goalkeeper,None,None,AchterbergJohn19710708
495,Adkins,Nigel,Nigel Adkins,1965-03-11,Goalkeeper,None,None,AdkinsNigel19650311
249,Ahmed,Adnan,Adnan Ahmed,1984-06-07,Central Midfield,None,None,AhmedAdnan19840607
92,Aimson,Will,Will Aimson,1994-06-03,Centre-Back,None,None,AimsonWill19940603
349,Aiston,Sam,Sam Aiston,1976-11-21,Left Midfield,None,None,AistonSam19761121
...,...,...,...,...,...,...,...,...
446,Yates,Steve,Steve Yates,1970-01-29,Centre-Back,Right-Back,Left-Back,YatesSteve19700129
577,Yeats,Ron,Ron Yeats,1937-11-15,Centre-Back,None,None,YeatsRon19371115
175,Young,Jack,Jack Young,2000-10-21,Central Midfield,Defensive Midfield,None,YoungJack20001021
583,Young,Tommy,Tommy Young,1947-12-24,Centre-Forward,Attacking Midfield,None,YoungTommy19471224


In [20]:
pos_df = pos_df[['player_id', 'surname', 'forename', 'player_name', 'player_dob', 'tm_pos_1', 'tm_pos_2', 'tm_pos_3']].sort_values(['player_id'])

pos_df.to_csv('./output/transfermarkt.csv', index=False)

In [21]:
r_ids = pd.read_csv('~/Desktop/player_ids.csv')[['player_id', 'player_name', 'player_dob']].drop_duplicates().reset_index(drop=True)

r_ids

,player_id,player_name,player_dob
0,ACourtAlan19340930,Alan A'Court,1934-09-30
1,AchterbergJohn19710708,John Achterberg,1971-07-08
2,AdamsArthur,Arthur Adams,NaN
3,AdamsEdward19081112,Edward Adams,1908-11-12
4,AdkinsNigel19650311,Nigel Adkins,1965-03-11
...,...,...,...
1020,YatesWilf1898,Wilf Yates,NaN
1021,YeatsRon19371115,Ron Yeats,1937-11-15
1022,YoungJack20001021,Jack Young,2000-10-21
1023,YoungTommy19471224,Tommy Young,1947-12-24


In [22]:
# Find players in Tranfermarkt data but not R dataset
# This mostly likely means players that did not make a first team appearance
errors = pos_df[~pos_df.player_id.isin(r_ids.player_id)][['player_id', 'player_name', 'player_dob']]

errors.sort_values(['player_dob'])

,player_id,player_name,player_dob
571,GriffithsJohn19160630,John Griffiths,1916-06-30
521,CurrieTony19500101,Tony Currie,1950-01-01
579,TynanTommy19551117,Tommy Tynan,1955-11-17
430,MiottoSimon19690905,Simon Miotto,1969-09-05
416,DavisNeil19730815,Neil Davis,1973-08-15
453,MoranAndy19791007,Andy Moran,1979-10-07
506,RalphAndrew19830528,Andrew Ralph,1983-05-28
509,RalphAndy19830528,Andy Ralph,1983-05-28
387,McGuireJamie19831113,Jamie McGuire,1983-11-13
358,BakerTom19850328,Tom Baker,1985-03-28


In [27]:
errors.merge(r_ids[['player_name', 'player_dob']], on='player_name', how='left') \
    .rename(columns={'player_dob_x': 'tm_dob', 'player_dob_y': 'r_dob'}) \
    .query("~r_dob.isnull()")

,player_id,player_name,tm_dob,r_dob
20,JonesEthan20041016,Ethan Jones,2004-10-16,1998-04-04
